In [0]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow_datasets as tfds
import sys
import h5py
import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
sys.path.append('/content/drive/My Drive/convAE_2_classifier')
from dataset import Dataset
from convAE import convAE

In [4]:
dataset = Dataset('cats_vs_dogs')
IMG_SIZE = 256
BATCH_SIZE = 32
input_shape = (IMG_SIZE, IMG_SIZE, 3)
dataset.set_img_size(IMG_SIZE)
train, val, test = dataset.format_data()
test_batches = test.batch(BATCH_SIZE)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteCBH4VC/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [0]:
# autoencoder_filters = [32, 64, 128, 256]
# filename = "convAE_stacked256_32-64-128-256__20200323-164116_Epoch0040-Loss0.0045_.hdf5"

# autoencoder_filters = [64, 128, 256, 512]
# filename = "convAE_stacked256_64-128-256-512__20200323-152310_Epoch0040-Loss0.0024_.hdf5"

autoencoder_filters = [64, 128, 256, 512, 1024]
filename = "convAE_stacked256_64-128-256-512-1024__20200323-163623_Epoch0040-Loss0.0039_.hdf5"

autoencoder = convAE(input_shape=input_shape)
weights_dir = "/content/drive/My Drive/convAE_2_classifier/weights/"
autoencoder.create_autoencoder_with_stacked_conv(filters=autoencoder_filters)
autoencoder.autoencoder.load_weights(weights_dir + filename)

In [150]:
classifier = load_model("/content/drive/My Drive/convAE_2_classifier/weights/classifier/mobileNet256_model.h5")
# classifier = load_model("/content/drive/My Drive/convAE_2_classifier/weights/classifier/denseNet169_256_weights.h5")

In [88]:
loss, acc = classifier.evaluate(test_batches)

73/73 [==============================] - 8s 103ms/step - loss: 0.1342 - accuracy: 0.9875


In [0]:
autoencoder.encoder.trainable = False
autoencoder.decoder.trainable = True
classifier.trainable = False

In [0]:
combinedModels = tf.keras.Sequential([autoencoder.autoencoder, classifier])

In [0]:
base_learning_rate = 0.00001
optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate)
combinedModels.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [154]:
loss, acc  = combinedModels.evaluate(test_batches)

73/73 [==============================] - 14s 190ms/step - loss: 2.2780 - accuracy: 0.7377


In [0]:
weights_dir = "/content/drive/My Drive/convAE_2_classifier/weights/"
filename = "convAE_stacked-MobielNet_3_256_64-128-256-512-1024__20200401-150919_Epoch0050-Loss0.0169_.hdf5"

# classifier_string = "convAE-denseNet/"
classifier_string = "convAE-mobileNet/"

# size = "small/"
# size = "medium/"
size = "big/"

combinedModels.load_weights(weights_dir + classifier_string + size + filename)

In [156]:
loss, acc  = combinedModels.evaluate(test_batches)

73/73 [==============================] - 14s 189ms/step - loss: 0.7259 - accuracy: 0.8534


# Reverse


In [0]:
autoencoder_filters = [32, 64, 128, 256]
filename = "convAE_stacked256_32-64-128-256__20200323-164116_Epoch0040-Loss0.0045_.hdf5"

# autoencoder_filters = [64, 128, 256, 512]
# filename = "convAE_stacked256_64-128-256-512__20200323-152310_Epoch0040-Loss0.0024_.hdf5"

# autoencoder_filters = [64, 128, 256, 512, 1024]
# filename = "convAE_stacked256_64-128-256-512-1024__20200323-163623_Epoch0040-Loss0.0039_.hdf5"

autoencoder = convAE(input_shape=input_shape)
weights_dir = "/content/drive/My Drive/convAE_2_classifier/weights/"
autoencoder.create_autoencoder_with_stacked_conv(filters=autoencoder_filters)
autoencoder.autoencoder.load_weights(weights_dir + filename)

In [141]:

mobileNet = load_model("/content/drive/My Drive/convAE_2_classifier/weights/classifier/mobileNet256_model.h5")

In [0]:
tmp_combined_models = tf.keras.Sequential([autoencoder.autoencoder, mobileNet])
# tmp_combined_models.load_weights("/content/drive/My Drive/convAE_2_classifier/weights/convAE_stacked-DenseNet256_32-64-128-256__20200325-224401_Epoch0050-Loss0.1170_.hdf5")

In [0]:
autoencoder.encoder.trainable = False
autoencoder.decoder.trainable = True
denseNet.trainable = False

In [0]:
weights_dir = "/content/drive/My Drive/convAE_2_classifier/weights/"
filename = "convAE_stacked-MobileNet_5_256_32-64-128-256__20200402-080710_Epoch0050-Loss0.0009_.hdf5"

# classifier_string = "convAE-denseNet/"
classifier_string = "convAE-mobileNet/"

size = "small/"
# size = "medium/"
# size = "big/"

tmp_combined_models.load_weights(weights_dir + classifier_string + size + filename)

In [0]:
denseNet = load_model("/content/drive/My Drive/convAE_2_classifier/weights/classifier/denseNet169_256_weights.h5")

In [0]:
reversed_combined_models = tf.keras.Sequential([autoencoder.autoencoder, denseNet])

In [0]:
base_learning_rate = 0.00001
optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate)
reversed_combined_models.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [148]:
reversed_combined_models.evaluate(test_batches)

73/73 [==============================] - 11s 146ms/step - loss: 1.0150 - accuracy: 0.8715


[1.014977216720581, 0.8714531660079956]